## 그룹 연산은 데이터를 집계하거나 변환하는 등의 작업을 한번에 처리할 수 있는 기능

### groupby 메서드로 평균값 구하기

In [1]:
import pandas as pd
df = pd.read_csv('../data/python_data/gapminder.tsv',sep='\t')
df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [2]:
#groupby 메서드를 사용 - year 열 기준, lifeExp 열의 평균값
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 분할 - 반영 - 결합 과정 살펴보기 - groupby 메서드

groupby 메서드 자체를 분해하는 것은 불가능하므로 비슷한 연산을 수행함

In [3]:
#years 중에 unique한 값들만
#groupby 메서드에 열 이름을 전달 하면 '분할' 작업이 일어남
years = df.year.unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

In [7]:
# 연도 하나만 가져오기
# 연도별로 데이터 추출 : 1952년을 추출한 예. '반영 - 1'
y1952 = df.loc[df.year == 1952, : ]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [8]:
# 추출한 데이터의 평균을 구함 '반영 - 2'
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

49.057619718309866


In [9]:
# 연도별 평균값을 구함 - '반영 - 3'
y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :]
y1962_mean = y1962.lifeExp.mean()
print(y1962_mean)

y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()
print(y2007_mean)

51.50740112676056
53.609249014084504
67.00742253521126


In [10]:
df2 = pd.DataFrame({"year":[1952,1957,1962,2007],
                   "":[y1952_mean, y1957_mean,y1962_mean,y2007_mean]})

print(df2)

   year           
0  1952  49.057620
1  1957  51.507401
2  1962  53.609249
3  2007  67.007423


## 평균값을 구하는 사용자 함수와 groupby 메서드
### 집계 기능을 내장하고 있는 판다스 기본함수
* mean(),max(),min(),sum(),count(),size(),var(),std(),describe(),info(),first(),last() 등
* agg() 메서드 데이터 집계 : group 객체.agg(매핑 함수)

In [11]:
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum+=value
    return sum/n

my_mean([1,2,4,7])

3.5

In [12]:
# 사용자 정의 함수(my_mean)를 groupby 메서드와 조합하기 위해 agg 메서드를 사용
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [30]:
# apply 메서드를 사용해도 같은 결과
agg_my_mean = df.groupby('year').lifeExp.apply(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [17]:
# 첫 번째 인자로 받은 열의 평균값을 구하여
# 두 번째 인자로 받을 값과의 차이를 계산하여 반환하는 함수 : my_mean_diff
def my_mean_diff(values, diff_value):
    n = len(values)  
    sum = 0
    for value in values:
        sum+=value
    mean = sum/n
    return mean - diff_value


my_mean_diff([1,2,3,4],10)

-7.5

In [18]:
# 전체 평균
global_mean = df.lifeExp.mean()
print(global_mean)

59.474439366197174


In [21]:
# 첫 번째 인자에 my_mean_diff, 두 번재 인자에 전체 평균 수명값을 전달
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff,diff_value = global_mean)

print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [37]:
df2 = pd.DataFrame({"year":years,
                   "연도별 평균 수명":agg_my_mean.values,
                   "전체 평균 수명":global_mean,
                   "결과":agg_mean_diff.values})

,year,연도별 평균 수명,전체 평균 수명,결과
0,1952,49.057620,59.474439,-10.416820
1,1957,51.507401,59.474439,-7.967038
2,1962,53.609249,59.474439,-5.865190
3,1967,55.678290,59.474439,-3.796150
4,1972,57.647386,59.474439,-1.827053
5,1977,59.570157,59.474439,0.095718
6,1982,61.533197,59.474439,2.058758
7,1987,63.212613,59.474439,3.738173
8,1992,64.160338,59.474439,4.685899
9,1997,65.014676,59.474439,5.540237


## 집계 메서드를 리스트, 딕셔너리에 담아 전달하기
- 모든 열에 여러 함수를 매핑 : group 객체.agg([함수1,함수2,함수3,...])
- 각 열마다 다른 함수를 매핑 : group 객체.agg({'열1':함수1, '열2':함수2,...})
- agg 함수는 집계가 목적이므로 데이터 타입이 숫자 타입인 행/열만 함수를 적용
<br>
<br>

### 집계 기능을 내앙하고 있는 판다스 기본 함수
- mean(),max(),min(),sum(),count(),size(),var(),std(),describe(),info(),first(),last() 등
- agg() 메서드 데이터 집계 : group 객체.agg(매핑 함수)

In [38]:
# 집계 매서드를 리스트에 담아 전달하기
# 연도별로 그룹화한 lieExp 열의  0이 아닌 값의 개수, 평균, 표준편차를 리스트에 담아 전달
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021


In [39]:
# 집계 메서드를 딕셔너리에 담에 agg 메서드에 전달
# 평균, 중간값(median)
gdf_dict = df.groupby('year').agg({'lifeExp':'mean','pop':'median','gdpPercap':'median'})

print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


## 데이터 변환

### 표준점수 계산하기

In [40]:
# transform 데이터 변환 메서드 - 데이터와 메서드를 일대일로 대응시켜 계산
# 데이터의 양이 줄어들지 않음
# 데이터 변환하는데 사용

def my_zscore(x):
    return (x-x.mean())/x.std()

transform_z = df.groupby('year').lifeExp.transform(my_zscore)

print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [41]:
df.shape

(1704, 6)

In [46]:
a = df.head(2)
print(a)
b = df.groupby('year').lifeExp.mean().head(2)
print(b)
c = df.groupby('year').lifeExp.std().head(2)
print(c)
print((28.801-49.057620)/12.225956)

       country continent  year  lifeExp      pop   gdpPercap
0  Afghanistan      Asia  1952   28.801  8425333  779.445314
1  Afghanistan      Asia  1957   30.332  9240934  820.853030
year
1952    49.057620
1957    51.507401
Name: lifeExp, dtype: float64
year
1952    12.225956
1957    12.231286
Name: lifeExp, dtype: float64
-1.656853664449635


In [42]:
# 데이터의 양이 줄어들지 않음을 확인(1704)
transform_z.shape

(1704,)

## 누락값을 평균값으로 처리하기

In [48]:
#sample()을 활용한 무작위 샘플 데이터 만들기
#무작위 표본 추출을 하는데 np.random.permutation() 함수를 사용하여 순열을 무작위로 섞은 후에 n개만큼 indexing 해오는 방법을 사용
# 'total_bill' 열의 값 4개를 임의 로 선택해서 누락값으로 변경

import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN

print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [49]:
# total_bill 누락값이 4개이므로 6개만 나옴, 다른 값은 10개
# total_bill 누락값 : 여성1남성3
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [50]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [53]:
# 성별을 구분하여 total_bill 열의 평균을 구하는 함수를 적용
# tips_10에 새로운 열
# 남성과 여성의 누락값을 고려하여 계산한 평균값으로 채움[남성->남성의 평균, 여성-> 여성의 평균]
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
    
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


## 데이터 필터링 사용하기 - filter 메서드

* 그룹 객체 필터링 : grop객체.filter(조건식 함수)

In [54]:
tips = sns.load_dataset('tips')
print(tips.shape)

(244, 7)


In [56]:
# size 열의 데이터 수를 확인
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
1      4
6      4
Name: size, dtype: int64


In [58]:
# 222 ->1, 216 ->5
tips.tail(n=30)

,total_bill,tip,sex,smoker,day,time,size
214,28.17,6.50,Female,Yes,Sat,Dinner,3
215,12.90,1.10,Female,Yes,Sat,Dinner,2
216,28.15,3.00,Male,Yes,Sat,Dinner,5
217,11.59,1.50,Male,Yes,Sat,Dinner,2
218,7.74,1.44,Male,Yes,Sat,Dinner,2
219,30.14,3.09,Female,Yes,Sat,Dinner,4
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
222,8.58,1.92,Male,Yes,Fri,Lunch,1
223,15.98,3.00,Female,No,Fri,Lunch,3


In [59]:
tips_filtered = tips.groupby('size').filter(lambda x:x['size'].count()>=30)
# lambda 인자: 표현식
print(tips_filtered.shape)

(231, 7)


In [60]:
# 222 ->1, 216 ->5  : 주문이 적은 1,5번 테이블을 제외하고 가져옴
tips_filtered.tail(30)

,total_bill,tip,sex,smoker,day,time,size
212,48.33,9.00,Male,No,Sat,Dinner,4
213,13.27,2.50,Female,Yes,Sat,Dinner,2
214,28.17,6.50,Female,Yes,Sat,Dinner,3
215,12.90,1.10,Female,Yes,Sat,Dinner,2
217,11.59,1.50,Male,Yes,Sat,Dinner,2
218,7.74,1.44,Male,Yes,Sat,Dinner,2
219,30.14,3.09,Female,Yes,Sat,Dinner,4
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
223,15.98,3.00,Female,No,Fri,Lunch,3


In [62]:
print(tips_filtered['size'].value_counts())

2    156
3     38
4     37
Name: size, dtype: int64


## 그룹 오브젝트 저장하여 살펴보기

In [65]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [66]:
# 자료형이 그룹 오브젝트임을 확인
grouped = tips_10.groupby('sex')
print(grouped)

In [67]:
# 그룹 오브젝트에 포함된 그룹을 보려면 groups 속성을 출력
print(grouped.groups)

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


## 그룹 오브젝트의 평균 구하기

In [68]:
tips_10.head()

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2


In [69]:
# sex, smoker, day, time 열은 그룹 연산에서 제외됨
# 파이썬은 그룹 연산에 적합한 열을 알아서 골라줌

avgs = grouped.mean()
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


C:\Users\Playdata\AppData\Local\Temp\ipykernel_3764\1701430143.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avgs = grouped.mean()


## 그룹 오브젝트에서 데이터 추출하고 반복하기

In [70]:
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [71]:
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


## 그룹 오브젝트 계산하고 살펴보기

In [72]:
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


C:\Users\Playdata\AppData\Local\Temp\ipykernel_3764\4174807147.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_avg = bill_sex_time.mean()


In [73]:
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [76]:
# reset_index 메서드를 사용하여 df에 새로운 인덱스 부여
group_method = tips_10.groupby(['sex','time']).mean().reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


C:\Users\Playdata\AppData\Local\Temp\ipykernel_3764\3042830927.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_method = tips_10.groupby(['sex','time']).mean().reset_index()


In [78]:
#reset_index -> as_index=False
group_params = tips_10.groupby(['sex','time'],as_index=False).mean()
print(group_params)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


C:\Users\Playdata\AppData\Local\Temp\ipykernel_3764\1197499840.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_params = tips_10.groupby(['sex','time'],as_index=False).mean()
